In [10]:
import pandas as pd

In [11]:
sales_bangalore_2022 = pd.read_csv('C:\\Users\\10684902\\Downloads\\Sales_Melbourne_2022.csv')               
display(sales_bangalore_2022)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-11-42b0a8198611>, line 1)

In [17]:
train_final = pd.read_csv('train_data_model.csv')
train_final.head(1)

,Unnamed: 0,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct
0,0,PRD0002-OL10330,0.6,0.004323,0.000404,1.0,0.000465


In [27]:
master_data = sales_bangalore_2022.copy()
master_data['trans_date'] = pd.to_datetime(master_data['trans_date'],format = '%m/%d/%Y')
master_data['start_date'] = pd.to_datetime(master_data['start_date'],format = '%m/%d/%Y')
master_data['mnth_code'] = pd.to_numeric(master_data['mnth_code'],errors ='coerce').astype(int)
df1=master_data.copy
master_data['unique_id']=master_data['product_code']+'-'+ master_data['outlet_code']

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_final[['frequencym','sales_value_avg','scheme_amount_perproduct']],train_final['sales_indicator'],test_size=0.3,random_state=42)

In [29]:
import pickle

with open('logreg.pkl', 'rb') as model_file:
   loaded_model = pickle.load(model_file)
probs = loaded_model.predict_proba(X_test)
probs

array([[0.66778068, 0.33221932],
       [0.8336751 , 0.1663249 ],
       [0.44592937, 0.55407063],
       ...,
       [0.66800724, 0.33199276],
       [0.83363294, 0.16636706],
       [0.6676696 , 0.3323304 ]])

In [30]:
cutoff = 0.3
y_pred = (probs[:,1]>=cutoff).astype(int)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

         0.0       0.83      0.60      0.70     30799
         1.0       0.57      0.82      0.68     20399

    accuracy                           0.69     51198
   macro avg       0.70      0.71      0.69     51198
weighted avg       0.73      0.69      0.69     51198



In [31]:
test_final = pd.read_csv('test_set_model.csv')

In [32]:
X_test_unseen = test_final[['frequencym','sales_value_avg','scheme_amount_perproduct']]
y_test_unseen = test_final['sales_indicator']
print(X_test_unseen.shape,y_test_unseen.shape)
probs = loaded_model.predict_proba(X_test_unseen)
test_final['probablity']=probs[:,1]
y_pred = (probs[:,1]>=0.4).astype(int)
test_final

(189082, 3) (189082,)


,Unnamed: 0,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct,probablity
0,0,PRD0002-OL10330,0.666667,0.004323,0.000404,1.0,0.001560,0.807117
1,1,PRD0002-OL10346,0.166667,0.001430,0.000135,0.0,0.001560,0.298748
2,2,PRD0002-OL10347,0.166667,0.001767,0.000168,0.0,0.001559,0.298690
3,3,PRD0002-OL10358,1.000000,0.015771,0.001481,1.0,0.001560,0.950099
4,4,PRD0002-OL10386,0.833333,0.008417,0.000791,1.0,0.001560,0.899377
...,...,...,...,...,...,...,...,...
189077,189077,PRD0168-OL97586,0.166667,0.001948,0.000303,1.0,0.001560,0.298657
189078,189078,PRD0168-OL97590,0.166667,0.000000,0.000000,0.0,0.001560,0.298999
189079,189079,PRD0168-OL97602,0.000000,0.001581,0.000242,0.0,0.001560,0.165792
189080,189080,PRD0168-OL97615,0.333333,0.001319,0.000202,1.0,0.001560,0.477312


In [33]:
test_final['prediction']= test_final['probablity'].apply(lambda x: 1  if x>=0.3 else 0)
test_final

,Unnamed: 0,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct,probablity,prediction
0,0,PRD0002-OL10330,0.666667,0.004323,0.000404,1.0,0.001560,0.807117,1
1,1,PRD0002-OL10346,0.166667,0.001430,0.000135,0.0,0.001560,0.298748,0
2,2,PRD0002-OL10347,0.166667,0.001767,0.000168,0.0,0.001559,0.298690,0
3,3,PRD0002-OL10358,1.000000,0.015771,0.001481,1.0,0.001560,0.950099,1
4,4,PRD0002-OL10386,0.833333,0.008417,0.000791,1.0,0.001560,0.899377,1
...,...,...,...,...,...,...,...,...,...
189077,189077,PRD0168-OL97586,0.166667,0.001948,0.000303,1.0,0.001560,0.298657,0
189078,189078,PRD0168-OL97590,0.166667,0.000000,0.000000,0.0,0.001560,0.298999,0
189079,189079,PRD0168-OL97602,0.000000,0.001581,0.000242,0.0,0.001560,0.165792,0
189080,189080,PRD0168-OL97615,0.333333,0.001319,0.000202,1.0,0.001560,0.477312,1


In [34]:
test_final_static = test_final.copy()
print(classification_report(test_final['sales_indicator'],test_final['prediction']))

              precision    recall  f1-score   support

         0.0       0.79      0.75      0.77    124905
         1.0       0.56      0.61      0.58     64177

    accuracy                           0.70    189082
   macro avg       0.67      0.68      0.68    189082
weighted avg       0.71      0.70      0.71    189082



In [40]:
test_final.reset_index(drop=True)
test_final['OOS'] = test_final['probablity'].apply(lambda x:1 if x>0.3 else 0)

In [41]:
soq_master = master_data.groupby('unique_id').agg({'sales_units':'sum','mnth_code':'nunique'})
soq_master['SOQ'] = soq_master['sales_units']/soq_master['mnth_code']
soq_master.reset_index()
final_reco = pd.merge(soq_master,test_final,on='unique_id',how='inner')

In [43]:
final_reco = final_reco[['unique_id','prediction','SOQ','OOS']]
final_reco.head()

,unique_id,prediction,SOQ,OOS
0,PRD0002-OL10330,1,2.714286,1
1,PRD0002-OL10346,0,1.500000,0
2,PRD0002-OL10347,0,1.250000,0
3,PRD0002-OL10358,1,7.166667,1
4,PRD0002-OL10386,1,4.090909,1


In [53]:
ms_reco = final_reco.copy()
ms_reco.columns = ['unique_id', 'ms_flag', 'soq', 'oos_flag']

In [54]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [55]:
ms_reco[['product', 'outlet']] = ms_reco['unique_id'].str.split('-', expand=True)
ms_reco.head()

,unique_id,ms_flag,soq,oos_flag,product,outlet
0,PRD0002-OL10330,1,2.714286,1,PRD0002,OL10330
1,PRD0002-OL10346,0,1.500000,0,PRD0002,OL10346
2,PRD0002-OL10347,0,1.250000,0,PRD0002,OL10347
3,PRD0002-OL10358,1,7.166667,1,PRD0002,OL10358
4,PRD0002-OL10386,1,4.090909,1,PRD0002,OL10386
